In [27]:
from array import array
from bisect import bisect_left as bl
from timeit import timeit
from tf.app import use
from pack import deepSize

In [32]:
class Sparse:
    def __init__(self, items):
        """Create sparse data container for (index, value) pairs.
        
        The pairs must be sorted by index, and no duplicate indices may occur.
        The set of indices must be a set of numbers, not necessarily consecutive.
        The values must be either all numbers, or all strings (Unicode UTF8).
        """
        
        if not items:
            self.isString = False
            self.indices = []
            self.values = []
            return
        
        isString = type(items[0][1]) is str
        self.isString = isString
        
        indices = []
        pointers = []
        values = []
        valueMap = {}
        curPos = 0
        
        if isString:
            curPos = 0
            for (k, v) in items:
                indices.append(k)
                if v in valueMap:
                    pointers.append(valueMap[v])
                else:
                    pointers.append(curPos)
                    vb = v.encode('utf8')
                    values.append(vb)
                    valueMap[v] = curPos
                    curPos += len(vb)
            pointers.append(curPos)
            self.values = b"".join(values)
        else:
            for (k, v) in items:
                indices.append(k)
                if v in valueMap:
                    pointers.append(valueMap[v])
                else:
                    pointers.append(curPos)
                    values.append(v)
                    valueMap[v] = curPos
                    curPos += 1
            self.values = array("I", values)
                
        self.indices = array("I", indices)
        self.pointers = array("I", pointers)
        
    def size(self):
        indices = self.indices
        pointers = self.pointers
        values = self.values
        
        iSize = indices.itemsize * len(indices)
        pSize = pointers.itemsize * len(pointers)
        vSize = len(values) if self.isString else (values.itemsize * len(values))
        return f"""{iSize + pSize + vSize} = 
indices:  {iSize:>8}
pointers: {pSize:>8}
values:   {vSize:>8}
"""
    
    def get(self, index):
        indices = self.indices
        if not indices:
            return None
        
        pick = bl(indices, index)
        if pick >= len(indices):
            return None
        where = indices[pick]
        if where != index:
            return None
        
        pointers = self.pointers
        pointer = pointers[pick]
        
        if self.isString:
            pointerNext = pointers[pick + 1]
            return self.values[pointer:pointerNext].decode('utf8')
            
        return self.values[pointer]

In [24]:
A = use('bhsa:clone', silent='deep')

In [33]:
# nametype
# sp

def testPerformance(feat):
    Fs = A.api.Fs
    featObj = Fs(feat)
    data = featObj.data
    
    dataS = Sparse(sorted(data.items()))
    
    print(f"{len(data)} items")
    print(f"Size (TF compiled) = {deepSize(data)}")
    print(f"Size (Sparse)      = {dataS.size()}")
    
    tfLookup = featObj.v
    spLookup = dataS.get
    
    def execute(v):
        upperIndex = 430000
        key0 = 739 # not in the data of nametype
        key1 = 740 # in the data of nametype
        
        def w():
            for i in range(700, 1700):
                x = v(i)
            
        def a():
            n = 0
            for i in range(upperIndex):
                if v(i) is not None:
                    n += 1
            
        times1 = 1000000
        times2 = 1000
        times3 = 1
        
        t1 = timeit("v(key0)", globals=locals(), number=times1)
        t2 = timeit("v(key1)", globals=locals(), number=times1)
        t3 = timeit("w()", globals=locals(), number=times2)
        t4 = timeit("a()", globals=locals(), number=times3)
        print(f"{t1:>.3f} {t2:>.3f} {t3:>.3f} {t4 * 1000000 / upperIndex:>.3f}")
    
    execute(tfLookup)
    execute(spLookup)

In [34]:
items = [(5, 500), (10, 1000)]
sn = Sparse(items)
for i in range(12):
    print(f"sn[{i}] = {sn.get(i)}")

sn[0] = None
sn[1] = None
sn[2] = None
sn[3] = None
sn[4] = None
sn[5] = 500
sn[6] = None
sn[7] = None
sn[8] = None
sn[9] = None
sn[10] = 1000
sn[11] = None


In [35]:
items = [(5, "foo ür bår"), (10, "𒉡𒊌=𒊍𒋫𒆷𒀠𒇷𒈬")]
ss = Sparse(items)
for i in range(12):
    print(f"ss[{i}] = {ss.get(i)}")

ss[0] = None
ss[1] = None
ss[2] = None
ss[3] = None
ss[4] = None
ss[5] = foo ür bår
ss[6] = None
ss[7] = None
ss[8] = None
ss[9] = None
ss[10] = 𒉡𒊌=𒊍𒋫𒆷𒀠𒇷𒈬
ss[11] = None


In [36]:
testPerformance('nametype')

38184 items
Size (TF compiled) = 2380461
Size (Sparse)      = 305536 = 
indices:    152736
pointers:   152740
values:         60

0.155 0.222 0.172 0.176
0.841 1.129 0.814 0.877


In [37]:
testPerformance('sp')

435817 items
Size (TF compiled) = 33175225
Size (Sparse)      = 3486595 = 
indices:   1743268
pointers:  1743272
values:         55

0.217 0.214 0.219 0.258
1.392 1.254 1.332 1.386
